# This notebook uses logistic regression to determine the impact of application responses on acceptance

In [309]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV
from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.graphics.gofplots import ProbPlot
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.utils import resample

from sklearn.metrics import roc_auc_score, classification_report

import seaborn as sns
import imblearn

## Load data

In [267]:
data = pd.read_csv("../data/clean/model_data.csv")

In [268]:
data_raw.head()

,ID Number,Preferred Pronouns,Current Grade Level,Program,First Paying Job,Know employee,Applied Before,Comfortable speaking to crowd,Uncomfortable with,Extra activities,...,Application Year,Current Age,Rejected,EECode,EarnHours,EarnRate,EarnAmount,Dist Position Desc,Year,Month
0,1489.0,masculine,9.0,zoocamp,1,0,0,1.0,invertebrates,0,...,2022,NaN,0,1489.0,5.50,13.0,71.50,WILD Steward,2022.0,4.0
1,1489.0,masculine,9.0,zoocamp,1,0,0,1.0,invertebrates,0,...,2022,NaN,0,1489.0,14.47,13.0,188.11,WILD Steward,2022.0,5.0
2,1489.0,masculine,9.0,zoocamp,1,0,0,1.0,invertebrates,0,...,2022,NaN,0,1489.0,7.13,13.0,92.69,WILD Steward,2022.0,5.0
3,1489.0,masculine,9.0,zoocamp,1,0,0,1.0,invertebrates,0,...,2022,NaN,0,1489.0,130.38,13.0,1694.94,WILD Steward,2022.0,6.0
4,1489.0,masculine,9.0,zoocamp,1,0,0,1.0,invertebrates,0,...,2022,NaN,0,1489.0,123.80,13.0,1609.40,WILD Steward,2022.0,7.0


## Define X, y and prepare data for modeling

In [269]:
data.columns

Index(['ID Number', 'Current Grade Level', 'Application Year', 'Current Age',
       'Rejected', 'EECode', 'EarnHours_Sum_AvgPerMonth',
       'Preferred Pronouns_feminine', 'Preferred Pronouns_masculine',
       'Preferred Pronouns_neutral', 'Preferred Pronouns_unknown',
       'Program_homeschool', 'Program_other', 'Program_unknown',
       'Program_zoocamp', 'First Paying Job_0', 'First Paying Job_1',
       'Know employee_0', 'Know employee_1', 'Applied Before_0',
       'Applied Before_1', 'Comfortable speaking to crowd_0.0',
       'Comfortable speaking to crowd_1.0',
       'Comfortable speaking to crowd_unknown', 'Uncomfortable with_birds',
       'Uncomfortable with_invertebrates', 'Uncomfortable with_mammals',
       'Uncomfortable with_none', 'Uncomfortable with_reptiles',
       'Uncomfortable with_unknown', 'Extra activities_0',
       'Extra activities_1', 'Commit Summer_unknown', 'Commit Summer_unsure',
       'Commit Summer_yes', 'Commit Weekday_no', 'Commit Weekday_unk

In [270]:
# Drop a column from one hot encoded values, specifically drop the _unknown columns where possible
# Pronouns have to be removed since they introduce perfect colinearity
X = data[['Current Grade Level', 'Application Year', 'Current Age','Program_homeschool', 'Program_other',
       'Program_zoocamp', 'First Paying Job_1',
       'Know employee_1',
       'Applied Before_1', 'Comfortable speaking to crowd_0.0',
       'Comfortable speaking to crowd_1.0',
       'Uncomfortable with_birds',
       'Uncomfortable with_invertebrates', 'Uncomfortable with_mammals',
       'Uncomfortable with_none', 'Uncomfortable with_reptiles',
       'Extra activities_1', 'Commit Summer_unsure',
       'Commit Summer_yes', 'Commit Weekday_no', 
       'Commit Weekday_unsure', 'Commit Weekday_yes',
       'Hear about source_family', 'Hear about source_friend',
       'Hear about source_other', 'Hear about source_school',
       'Hear about source_social media']]

y = data['Rejected']

In [271]:
# Identify perfect multicollinearity
for i,a in enumerate(X.columns):
    for j,b in enumerate(X.columns):
        k = sum(X[X.columns[i]]+X[X.columns[j]])
        if k == len(X):
            print(f"{a}, {b}: {k}")

First Paying Job_1, Know employee_1: 289
Know employee_1, First Paying Job_1: 289


In [272]:
X = X.drop(columns = ['Know employee_1'])

In [273]:
X

,Current Grade Level,Application Year,Current Age,Program_homeschool,Program_other,Program_zoocamp,First Paying Job_1,Applied Before_1,Comfortable speaking to crowd_0.0,Comfortable speaking to crowd_1.0,...,Commit Summer_unsure,Commit Summer_yes,Commit Weekday_no,Commit Weekday_unsure,Commit Weekday_yes,Hear about source_family,Hear about source_friend,Hear about source_other,Hear about source_school,Hear about source_social media
0,9.0,2022,15.339623,0,0,1,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
1,11.0,2022,15.339623,0,1,0,0,0,0,1,...,1,0,0,0,1,0,0,1,0,0
2,11.0,2022,15.339623,0,1,0,0,0,0,1,...,0,1,0,0,1,0,1,0,0,0
3,9.0,2022,15.339623,0,1,0,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
4,10.0,2022,15.339623,0,1,0,0,0,0,1,...,1,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,11.0,2020,16.000000,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,1,0
285,10.0,2020,15.000000,0,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
286,10.0,2020,15.000000,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,0
287,10.0,2020,15.000000,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,0,1,0


## Check results of logistic regression

In [274]:
# with statsmodels 
model = sm.Logit(y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

         Current function value: 0.313787
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:               Rejected   No. Observations:                  289
Model:                          Logit   Df Residuals:                      263
Method:                           MLE   Df Model:                           25
Date:                Tue, 21 Feb 2023   Pseudo R-squ.:                  0.1799
Time:                        23:11:52   Log-Likelihood:                -90.684
converged:                      False   LL-Null:                       -110.58
Covariance Type:            nonrobust   LLR p-value:                   0.03068
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Current Grade Level                   0.0358      0.263      0.136      0.892      -0.480       0.

C:\Users\lawto\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


## Is a regularized model better?

In [279]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [280]:
model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [281]:
def get_model_preds(X, betas):
    xtb = np.dot(X, betas)
    probs = 1 / (1+np.exp(-xtb))
    preds = (probs > 0.5)*2-1 # convert preds to +/- 1
    return probs, preds

def acc(preds, truth):
    return np.mean(preds==truth)

In [284]:
preds = model.predict(X_train)
preds_test = model.predict(X_test)

print(f"baseline prediction: {np.mean(y_test==1)}") # constant predictor
print(f"train accuracy: {acc(preds, y_train)}")
print(f"test accuracy: {acc(preds_test, y_test)}")

baseline prediction: 0.8620689655172413
train accuracy: 0.8744588744588745
test accuracy: 0.8620689655172413


In [286]:
# check auc score
roc_auc_score(y_test, model.predict_proba(X_test)[:,1])

0.635

## what if we tune our hyperparameter?

In [290]:
cvres = LogisticRegressionCV(Cs=50).fit(X_train,y_train)
C = cvres.C_[0]
C

C:\Users\lawto\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\lawto\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

0.0001

In [291]:
lr = LogisticRegression(C = C, solver='liblinear')
lr.fit(X_train, y_train)

LogisticRegression(C=0.0001, solver='liblinear')

In [293]:
preds = lr.predict(X_train)
preds_test = lr.predict(X_test)

print(f"baseline prediction: {np.mean(y_test==1)}") # constant predictor
print(f"train accuracy: {acc(preds, y_train)}")
print(f"test accuracy: {acc(preds_test, y_test)}")

baseline prediction: 0.8620689655172413
train accuracy: 0.8744588744588745
test accuracy: 0.8620689655172413


In [294]:
# check auc score
roc_auc_score(y_test, model.predict_proba(X_test)[:,1])

# the model always predicts 1...

0.635

## Use SMOTE to address the fact that we have a huge class imbalance, and very few observations in the positive class

In [319]:
# use SMOTE to balance classes
oversample = imblearn.over_sampling.SMOTE(random_state = 42)
X_train_o, y_train_o = oversample.fit_resample(X_train, y_train)

print(f"total synthetic obs: {len(y_train_o)}")
print(f"positive synthetic obs: {sum(y_train_o)}")

total synthetic obs: 404
positive synthetic obs: 202


In [320]:
# hyperparameter tune
cvres_o = LogisticRegressionCV(Cs=50).fit(X_train_o,y_train_o)
C = cvres_o.C_[0]
print(f"best C: {C}")

# fit model
lr_best = LogisticRegression(C = C, solver='liblinear')
lr_best.fit(X_train_o, y_train_o)

C:\Users\lawto\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\lawto\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

best C: 16.768329368110066


C:\Users\lawto\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\lawto\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

LogisticRegression(C=16.768329368110066, solver='liblinear')

In [324]:
preds = lr_best.predict(X_train_o)
preds_test = lr_best.predict(X_test)

print(f"baseline prediction: {np.mean(y_test==1)}") # constant predictor
print(f"train accuracy: {acc(preds, y_train_o)}")
print(f"test accuracy: {acc(preds_test, y_test)}")

baseline prediction: 0.8620689655172413
train accuracy: 0.7945544554455446
test accuracy: 0.8103448275862069


In [326]:
roc_auc_score(y_test, lr_best.predict_proba(X_test)[:,1])

0.445